# Project 2 - Ames Housing Data and Kaggle Challenge
---

## Library and Data Imports

In [1]:
## library imports

# data processing imports
import pandas as pd
import numpy as np

# modelling imports
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn import linear_model

In [2]:
pd.set_option('display.max_columns', None) # disable column overflow in pandas display, this is toenable us to display all the columns

In [3]:
## Data import
df = pd.read_csv('../datasets/test.csv') # importing training data set
df.head() # display the head

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemod/Add,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,NaN,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,NaN,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,NaN,NaN,NaN,0,7,2009,WD


## Data Cleaning

### Deleting Index (ID) columns

Identification (ID) columns should be deleted as they provide no predictive capability

In [4]:
df.drop(['PID'],axis=1,inplace=True)

### Checking for Null Values

In [5]:
nulls = pd.DataFrame() # instantiate dataframe to store info on null values in different columns
nulls['no_of_null'] = df.isnull().sum() # calculate the number of nulls in each column
nulls['ratio_of_null'] = nulls['no_of_null'] / df.shape[0] # calculate the ratio of nulls to number of rows in each column
nulls[~(nulls==0).all(axis=1)] # show all columns with null values

,no_of_null,ratio_of_null
LotFrontage,160,0.182232
Alley,820,0.933941
MasVnrType,1,0.001139
MasVnrArea,1,0.001139
BsmtQual,25,0.028474
BsmtCond,25,0.028474
BsmtExposure,25,0.028474
BsmtFinType1,25,0.028474
BsmtFinType2,25,0.028474
Electrical,1,0.001139


### Dealing with Null Values

#### Columns with High Ratio (> 90%) of Null Values

In [6]:
nulls[nulls['ratio_of_null'] > 0.9] # displaying columns with high ratio of null values

,no_of_null,ratio_of_null
Alley,820,0.933941
PoolQC,874,0.995444
MiscFeature,837,0.953303


Typically these columns with high null ratios should be dropped. However, according to the data documentations, the null values here refer to the absence of such feature. As such we will impute the null values as the string 'NA'.

In [7]:
cols_to_impute = nulls[nulls['ratio_of_null'] > 0.9].index # getting the column names to be dropped
df[cols_to_impute] = df[cols_to_impute].fillna('NA') # imputing the columns based on list above

#### Columns with Low Ratio (< 10%) of Null Values

Generally columns with low ratio of null values can be imputed. The following sections discusses the various strategies used for Imputation on the various columns:

##### Garage-Related Columns

The NaN values in Garage is due to the absence of garage in the house. Hence we can impute the numerical values as 0, and create a new category for all the categorical variables which states 'No Garage'

In [8]:
garage_cols = [col for col in nulls.index if 'Garage' in col] # selecting all the columns that contains the word 'Garage'
df[garage_cols].dtypes # testing the data type of the garage-related columns

GarageType       object
GarageYrBlt     float64
GarageFinish     object
GarageCars        int64
GarageArea        int64
GarageQual       object
GarageCond       object
dtype: object

In [9]:
garage_cols_categorical = df[garage_cols].select_dtypes('object').columns # selecting garage-related columns with categorical values
df[garage_cols_categorical] = df[garage_cols_categorical].fillna('NA') # imputing the columns with 'NA'

In [10]:
df[['GarageCars','GarageArea']] = df[['GarageCars','GarageArea']].fillna(0) # imputing the numerical columns with 0 (no garage)

Unlike the no of cars and Garage Area, we would not be able to impute the year with 0, instead, we can impute the mean value for the year (will be done in later cells)

##### Remaining Columns

In [11]:
cols_to_impute = nulls[(nulls['ratio_of_null'] < 0.1) & (~(nulls==0).all(axis=1))].index # getting the column names to be imputed

In [12]:
# imputing numeric columns with column-wise mean
cols_to_impute_numeric = df[cols_to_impute].select_dtypes(np.number).columns # getting numeric column names to be imputed
df[cols_to_impute_numeric] = df[cols_to_impute_numeric].fillna(df[cols_to_impute_numeric].mean()) # imputing the numeric columns using its column-wise mean

In [13]:
# imputing categoric columns with column-wise mode
cols_to_impute_categoric = df[cols_to_impute].select_dtypes(['object']).columns # getting categoric column names to be imputed

for column in cols_to_impute_categoric: # imputing the categoric columns using its mode
    column_mode = df[column].mode()[0] # getting the column-wise mode
    df[column].fillna(column_mode,inplace=True) # imputing the NaN values with mode

#### Remainder of Columns

In [14]:
nulls[(nulls['ratio_of_null'] >= 0.1) & (nulls['ratio_of_null'] <= 0.9)]

,no_of_null,ratio_of_null
LotFrontage,160,0.182232
FireplaceQu,422,0.480638
Fence,706,0.804100


##### Lot Frontage

Since there are 60% of the data missing we need to use a more advanced method for imputation

In [15]:
# using simple linear regression model for imputing the value of Lot Frontage based on Lot Area

# importing models from sklearn
from sklearn import linear_model

df_impute = df[['LotArea','LotFrontage']] # create dataframe for imputing
df_impute = df_impute.dropna() # drop the NaN values from imputation dataframe

# # create a Linear Regression model to estimate the missing data
impute_model = linear_model.LinearRegression() # instantiate LR model
impute_model.fit(X = df_impute['LotArea'].values.reshape(-1, 1), y = df_impute['LotFrontage'].values) # fitting the model based on the imputation dataframe

df['LotFrontageIMPUTED'] = impute_model.predict(df['LotArea'].values.reshape(-1, 1)) # getting the imputation value from the impute model

df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontageIMPUTED']) # imputing the missing value in Lot Frontage

df.drop('LotFrontageIMPUTED',axis=1,inplace=True)# dropping the IMPUTED column

##### Fireplace Qu

In [16]:
# looking at the fireplace columns
df[['Fireplaces','FireplaceQu']].head()

,Fireplaces,FireplaceQu
0,0,NaN
1,0,NaN
2,1,Gd
3,0,NaN
4,2,Gd


It seems that the null values correspond to rows with no fireplaces

In [17]:
df['FireplaceQu'] = df['FireplaceQu'].fillna('NA') # filling the NaN values with the text 'NoFireplace

##### Fence

In [18]:
df['Fence'] = df['Fence'].fillna('NA') # filling the NaN values with the text 'NoFence'

#### Check that there is no more nan values

In [19]:
df.isna().sum().sum()

0

There are no more null values in the dataframe

## Setting Up Features and Target Variables

In [20]:
X = df

### Converting Ordinal Categorical Variables

In [21]:
grade_columns = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond','PoolQC'] # storing the columns which contains grading
grade_scoring = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5} # assinging each grade an ordinal score

for column in grade_columns:
    X[column] = X[column].map(grade_scoring) # replacing the grades with the scoring

Based on the [data documentation](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt), there are other ordinal columns in the dataset. The following cells will deal with each of the ordinal columns, assinging the score to each of the orginal columns.

In [22]:
X['LotShape'] = X['LotShape'].map({'IR3':0,'IR2':1,'IR1':2,'Reg':3})
X['Utilities'] = X['Utilities'].map({'NoSeWa':0,'NoSewr':1,'AllPub':2})
X['LandSlope'] = X['LandSlope'].map({'Sev':0,'Mod':1,'Gtl':2})
X['BsmtExposure'] = X['BsmtExposure'].map({'Gd':0,'Av':1,'Mn':2,'No':3})
X['BsmtFinType1'] = X['BsmtFinType1'].map({'Unf':0,'LwQ':1,'Rec':2,'BLQ':3,'ALQ':4,'GLQ':5})
X['BsmtFinType2'] = X['BsmtFinType2'].map({'Unf':0,'LwQ':1,'Rec':2,'BLQ':3,'ALQ':4,'GLQ':5})
X['Electrical'] = X['Electrical'].map({'Mix':0,'FuseP':1,'FuseF':2,'FuseA':3,'SBrkr':4})
X['Functional'] = X['Functional'].map({'Sal':0,'Sev':1,'Maj2':2,'Maj1':3,'Mod':4,'Min2':5,'Min1':6,'Typ':7})
X['GarageFinish'] = X['GarageFinish'].map({'NA':0,'Unf':1,'RFn':2,'Fin':3})
X['PavedDrive'] = X['PavedDrive'].map({'N':0,'P':1,'Y':2})
X['Fence'] = X['Fence'].map({'NA':0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':0})

The remainder columns are all nominal categorical variables and can then be one-hot encoded

### One-Hot Encoding for Nominal Categorical Variables

In [23]:
X_dummies = pd.get_dummies(X.select_dtypes('object'),drop_first=True) # getting the dummy variables from only the categorical variables
X = X.drop(X.select_dtypes('object').columns,axis=1) # dropping the columns with categorical variables
X = pd.concat([X,X_dummies],axis=1) # concatenating the numerical X matrix (X) and the dummified categorical X matrix (X_dummies)

### Dropping Garage Cars to prevent Multi-colinearity

In [24]:
X.drop('GarageCars',axis=1,inplace=True) # dropping garaga cars as it is multi-colinear with GarageArea

### Adding non-linearity to Conditions and Qualitioes

In [25]:
conditions_columns = ['OverallCond','ExterCond','BsmtCond','HeatingQC','GarageCond','PoolQC']
quality_columns = ['OverallQual','BsmtQual','KitchenQual','ExterQual','FireplaceQu','GarageQual']

In [26]:
X[quality_columns] = X[quality_columns]**2
X[conditions_columns] = np.log(X[conditions_columns]+1)

In [27]:
X.to_csv('../datasets/test_processed.csv',index=False)